## Imports and globals

Here you will have to set you own `apikey` by replacing the `enter-your-api-key-here` text.

In [1]:
import requests
import json
import sys

# Change apikey with the apikey retrieved from CloudOS' webpage
apikey = "enter-your-api-key-here"

# This is the url for CloudOS service. Do not change it!
cloudos_url = "https://cloudos.lifebit.ai"

## Fetch project id

In order to run any job in CloudOS you need to have a project to which you will associate your jobs. Therefore, you need to retrieve the project id for the project name in which you want to associated the job that you are running. For example, you may want to run the job within the `demo` project!

In [2]:
# This function will fetch the project id for a given project name
def fetch_cloudos_project_id(apikey, project_name):
    data = {"apikey": apikey}
    r = requests.get("{}/api/v1/projects".format(cloudos_url), params=data)
    for project in json.loads(r.content):
        if project["name"] == project_name:
            return project["_id"]
    

In [3]:
project_id = fetch_cloudos_project_id(apikey, "Demo")
if not project_id:
    sys.exit("Project id couldn't be find. Check that you provided a valid project name")

## Fetch workflow id

In order to fetch workflow id you will need to know the workflow name that you are looking for in CloudOS and then you can request the workflow id to our api with the following snippet:

In [4]:
# This function will fetch the workflow id for a given workflow name
def fetch_cloudos_workflow_id(apikey, workflow_name):
    data = {"apikey": apikey}
    r = requests.get("{}/api/v1/workflows".format(cloudos_url), params=data)
    for workflow in json.loads(r.content):
        if workflow["name"] == workflow_name:
            return workflow["_id"]

In [5]:
workflow_id = fetch_cloudos_workflow_id(apikey, "GWAS")
if not workflow_id:
    sys.exit("workflow id couldn't be find. Check that you provided a valid workflow name")

## Running the job and checking it's status

The following snippet takes the ids fetch above and makes a request to CloudOS api:

In [6]:
def send_job_request_to_cloudos(apikey, workflow_id, project_id):
    # Prepare api request for CloudOS to run a job
    headers = {
        "Content-type": "application/json",
        "apikey": apikey
    }

    workflow_params = [
       {
         "name": "input_param",
         "prefix": "--",
         "parameterKind": "textValue",
         "textValue": "some value"
       }
    ]

    params = {
        "executionPlatform": "aws",
        "instanceType": "m2.2xlarge",
        "parameters": workflow_params,
        "project": project_id,
        "workflow": workflow_id
    }

    r = requests.post("{}/api/v1/jobs".format(cloudos_url), data=json.dumps(params), headers=headers)
    return json.loads(r.content)["_id"]

In [7]:
job_id = send_job_request_to_cloudos(apikey, workflow_id, project_id)
if job_id:
    print("Job successfully sent to CloudOS. You can check the status " +
          "of the job in https://cloudos.lifebit.ai/app/jobs/{}".format(job_id))

Job successfully sent to CloudOS. You can check the status of the job in https://cloudos.lifebit.ai/app/jobs/job_id
